### Метод Гомари

c`x -> max
Ax = b
x >= 0
x - целые

В базисном > m не0 компонент Значит > m ограничений

Задача: построение дополнительных ограничений позволяющих отсечь нецелые решения

Выпуклая оболчка - x_b

$x_{цел} \in x_b \in x$

Условия:
* отсекать нецелочисленные решения
* не отсекать ни одного целочисленного решения
* отсечение должно быть посторено так, чтобы являтся конечным по кол-ву задач. Т.е. через некоторое чсило шагов, должен
  быть найден ответ

Предположим что ограничения совместны и задача ограничена сверху
Составление доп задач
1. Рассмотрим линейные комбинации уравнения Ax = b:
y'Ax=y'b

y - произвольный вектор

Обозначим: $a_j = y' A_j, \beta = y'b$

Тогда уравнение примет вид: $\sum_{j=1} a_j x_j = \beta$

Т.к. все компоненты > 0:

$\sum [a_j] x_j <= beta$

$\sum [a_j] x_j <= [beta]$

Перепшем в виде равентсва
\Sum [a_j] x_j + x_* = [beta]

$f = beta - [beta]$

$f_j = a_j - [a_j]$

(3) = (1) - (2): $\sum [-f_j] x_j + x_* = -f$

 Шаги:
 1. Решить задачу симплекс методом - если есть int ответ то задача решена
 2. Выбираем базисную нецелочисленную компоненту для постоения ограничений:

$x_i^0 $ -> $ y' = e_i^0 A_B^{-1}$

  3. Формируем (3), добавления отсекающее ограничение и новую целочисл переменную в задачу


## Пример

$21x_1 + 11x_2 \rightarrow max$

$7x_1 + 4x_2 + x_3 = 13$

$x_j >= 0 j \in [1,3]$


Решение задачи:

$x_1^0 = \dfrac{13}{7} , x_2^0 = x_3^0 = 0$
$J_b = {1}$

Выбор переменной:

$i_0 = 1$
$ A_b = 7, A_B^{-1} = \dfrac{1}{7} $
$y' = e_{i_0} \cdot A_B^{-1} = \dfrac{1}{7}$

Итого:

$21x_1 + 11_x \rightarrow max$

$7x_1 + 4x_2 + x_3 = 13$

$-\dfrac{4}{7}x_2 - \dfrac{1}{7}x_3 + x_4 = -\dfrac{6}{7} $

$x_j >= 0 j \in [1,3]$

$x_4 > 0$

Итерации повторяются аналогично, до достижения результата:

$x_1 = 0, x_2 = 3, x_3 = 1, x_4=1, x_5=0, J_B = {2,3,4}$

In [376]:

import numpy as nmp
from simplex import simplex_min, simplex_max
from scipy.optimize import linprog

In [377]:
def two_step_simplex_max(a: nmp.ndarray, b: nmp.ndarray, c: nmp.ndarray):
    ext_size = a.shape[0]
    a_ext = nmp.concatenate((a, nmp.eye(ext_size)), axis=1)
    x_ext = nmp.zeros(len(c) + ext_size)
    x_ext[-ext_size:] = b
    c_ext = nmp.concatenate((c, nmp.zeros(ext_size)))
    j_b_ext = nmp.arange(ext_size) + len(c)
    return simplex_max(a_ext, c_ext, x_ext, j_b_ext)


def fractional(f: float) -> float:
    if f < 0:
        return f - int(f) + 1.
    else:
        return f - int(f)


def get_basis(x: nmp.ndarray, size: int) -> nmp.ndarray:
    b = nmp.array(nmp.nonzero(x))[0]
    if len(b) > size:
        b = b[:size]
    elif len(b) < size:
        b = nmp.concatenate((
            b,
            nmp.setdiff1d(nmp.arange(len(x)), b)[:size - len(b)]
        ))
    return b

In [378]:
# define test task

o_a = nmp.array([
    [10, 30],
    [25, 25],
    [41, 90],
    [90, 50],
])
o_b = nmp.array([4500, 6250, 14100, 18000])

o_c = nmp.array([100, 250])
o_x = nmp.array([0, 0])

# to canonical form

c_count = o_a.shape[0]
t_a = nmp.concatenate((o_a, nmp.eye(c_count)), 1)
t_c = -nmp.concatenate((o_c, nmp.zeros(c_count)))
t_b = o_b
t_x = nmp.concatenate((o_x, o_b))

In [379]:
#define ex4:
# t_a = nmp.array([
#     [3, 4, 1, 0],
#     [2, 5, 0, 1],
# ])
# o_c = [2, 3]
# t_c = -nmp.array([2, 3, 0, 0])
# t_b = nmp.array([24, 22])
# t_x = nmp.array([0, 0, 24, 22])

In [380]:

#define ex2:
# t_a = nmp.array([
#     [-1, 3, 1, 0],
#     [7, 1, 0, 1],
# ])
# o_c = [7 , 9]
# t_c = -nmp.array([7, 9, 0, 0])
# t_b = nmp.array([6 , 35])
# t_x = nmp.array([0 , 0 , 6 , 35])

In [381]:
def get_eq(a: nmp.ndarray, x: nmp.ndarray, j_b: nmp.ndarray) -> tuple[nmp.ndarray, float]:
    k = nmp.argmax([round(fractional(f), 4) for f in x[j_b]])
    j_n = nmp.setdiff1d(nmp.arange(len(x)), j_b)
    l = nmp.linalg.inv(a[:, j_b]).dot(a[:, j_n])[k]
    eq = nmp.zeros(len(x) + 1)
    eq[-1] = -1
    for i, v in zip(j_n, l):
        eq[i] = fractional(v)
    eq = -eq
    nb = -fractional(x[j_b[k]])
    print(j_b[k])
    print(eq, nb)
    return eq, nb

In [382]:
# new_eq, new_b = get_eq(c_a, rx, rjb)
# t_b = nmp.append(o_b, [new_b])
# t_a = nmp.concatenate((c_a, nmp.zeros((c_a.shape[0], 1))), 1)
# t_a = nmp.concatenate((t_a, [new_eq]))
# t_c = nmp.concatenate((c_c, [0]))

In [383]:
start_count = len(o_c)

while True:

    rx = linprog(t_c, None, None, t_a, t_b, method='revised simplex').x
    rjb = get_basis(rx, t_a.shape[0])
    print(rx, rjb)
    if all([round(f, 4).is_integer() for f in rx[:start_count]]):
        print(f'Результат получен: {rx[:start_count]}')
        break

    new_eq, new_b = get_eq(t_a, rx, rjb)
    t_c = nmp.concatenate((t_c, [0]))
    t_b = nmp.append(t_b, [new_b])
    t_a = nmp.concatenate((t_a, nmp.zeros((t_a.shape[0], 1))), 1)
    t_a = nmp.concatenate((t_a, [new_eq]))



[4.5 3.5 0.  0. ] [0 1]
0
[-0.         -0.         -0.95454545 -0.13636364  1.        ] -0.5
[4.52380952 3.33333333 0.52380952 0.         0.        ] [0 1 2]
0
[-0.         -0.         -0.         -0.14285714 -0.95238095  1.        ] -0.5238095238095237
[4.55 3.15 1.1  0.   0.55 0.  ] [0 1 2 4]
0
[-0.   -0.   -0.   -0.15 -0.   -0.95  1.  ] -0.5499999999999998
[4.57894737 2.94736842 1.73684211 0.         1.15789474 0.57894737
 0.        ] [0 1 2 4 5]
1
[-0.         -0.         -0.         -0.89473684 -0.         -0.
 -0.36842105  1.        ] -0.9473684210526319
[4.41176471 3.05882353 1.23529412 1.05882353 0.82352941 0.41176471
 0.         0.        ] [0 1 2 3 4 5]
4
[-0.         -0.         -0.         -0.         -0.         -0.
 -0.76470588 -0.35294118  1.        ] -0.8235294117647055
[4.00000000e+00 3.33333333e+00 4.44089210e-16 3.66666667e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.33333333e+00
 0.00000000e+00] [0 1 2 3 7 4 5]
0
[-0.  -0.  -0.  -0.  -0.  -0.  -0.5 -0.  -0.5 

In [384]:
# ex1: